In [1]:
import numpy as np
def extEucl(m, n):
    """
    Возвращает тройку (d,u,v) таких, что d — наибольший общий делитель пары (m,n) и d=um+vn
    """
    (a, b) = (m, n)
    u1 = 1; v1 = 0
    u2 = 0; v2 = 1

    while b != 0:
        assert (a == u1*m + v1*n and b == u2*m + v2*n)

        q = a // b; r = a % b
        assert (a == q*b + r)

        (a, b) = (b, r)
        (u1, u2) = (u2, u1 - q*u2)
        (v1, v2) = (v2, v1 - q*v2)

    if a >= 0:
        return (a, u1, v1)
    else:
        return (-a, -u1, -v1)
    
def invmod(x, m):
    """
    Возвращает обратный к x элемент, если x обратим (т.е. целые числа x,m взаимно простые),
    иначе - исключение типа ValueError
    """
    assert(m != 0)
    (d, u, v) = extEucl(m, x)
    if d == 1:
        return v%m
    else:
        raise ValueError("Element is not invertible modulo m")

## Алиса формирует открытый и закрытый ключи
$r = (r_1, . . . , r_n)$ - супер последовательность

In [2]:
r = np.array((3, 11, 24, 50, 115))

Выбираем $A$ и $B$ такие, что $B>2r_n$ и $(A,B)=1$

In [3]:
A = 113
B = 250

Вычисляем $M_i = Ar_i \pmod{B}$ для $i\in[1,n]$

In [4]:
M=np.array(list(map(lambda x: (x*A)%B,r)))
assert np.array_equal(M, np.array([89, 243, 212, 150, 245])) 

#### Публичный ключ Алисы - $M$
## Боб формирует сообщение
Секретное сообщение $x$

In [5]:
x = np.array((1, 0, 1, 0, 1))
x

array([1, 0, 1, 0, 1])

Расчитываем $$S = x * M$$

In [6]:
S = (x * M).sum()
assert S==546

###### Отправляем $S$ по каналу связи
## Алиса расшифровывает послание

Вычисляем $A^{-1}$ по модулю $B$

In [7]:
A_=invmod(A,B)
assert A_== 177

Вычисляем $$S^′ ≡ A^{−1}S (mod B)$$

In [8]:
S_=(A_*S)%B
assert S_==142

Затем решаем $$S^′ = x * r$$ где $x$ - исходное сообщение

In [9]:
def find_x(r,S):
    x=np.zeros(r.shape)
    for i in range(len(r)-1,-1,-1):
        if S>=r[i]:
            x[i]=1
            S-=r[i]
        else:
            x[i]=0
    return x.astype("int")

In [10]:
x=find_x(r,S_)
print("Передаваемое сообщение:",x)

Передаваемое сообщение: [1 0 1 0 1]


## Вмешивается Ева
Формируем матрицу вида:
$$v_1 = (2, 0, 0, . . . , 0, m_1),$$
$$v_2 = (0, 2, 0, . . . , 0, m_2),$$
$$... ...$$
$$v_n = (0, 0, 0, . . . , 2, m_n),$$
$$v_{n+1} = (1, 1, 1, . . . , 1, S)$$

In [11]:
V=np.array([[ 1 if i==len(M) else (2 if i==j else 0)  for j in range(len(M))] for i in range(len(M)+1)])
last_col=np.array([S if i==len(M) else M[i] for i in range(len(M)+1)]).reshape(-1,1)
V = np.hstack([V, last_col])
V

array([[  2,   0,   0,   0,   0,  89],
       [  0,   2,   0,   0,   0, 243],
       [  0,   0,   2,   0,   0, 212],
       [  0,   0,   0,   2,   0, 150],
       [  0,   0,   0,   0,   2, 245],
       [  1,   1,   1,   1,   1, 546]])

Получим решётку $L$
$$L = \{a_1v_1 + a_2v_2 + · · · + a_nv_n + a_{n+1}v_{n+1} : a_1, a_2, . . . , a_{n+1} ∈ Z\}$$
Предположив, что $x = (x_1, . . . , x_n)$ - решение данной задачи:
$$t =\displaystyle\sum_{i=1}^{n} x_iv_i − v_{n+1} = (2x_1 − 1, 2x_2 − 1, . . . , 2x_n − 1, 0),$$

Применяем алгоритм LLL для редукции базиса

In [12]:
def gram_schmid(b):
    b_=np.zeros(b.shape)
    m=np.zeros(b.shape)
    b_[0]=b[0]
    for i in range(1,b.shape[1]):
        v=b[i]
        for j in range(i-1,-1,-1):
            m[i][j]=np.dot(b[i],b_[j])/np.dot(b_[j],b_[j])
            v=v-m[i][j]*b_[j]
        b_[i]=v
    return b_.astype("int"),m
norma = lambda x: np.power(math.sqrt(np.array([i ** 2 for i in x]).sum()),2)
def LLL(b, delta=0.75):
    b_, m = gram_schmid(b)
    B = np.array([norma(i) for i in b_])
    q = np.zeros(b.shape[1])
    n = b.shape[1]-1
    k = 1
    old_b=b.copy()
    old_b_ = b_.copy()
    counter=0
    while k <= n:
        for j in range(k-1,-1,-1):
            q[j] = round(m[k][j])
            b[k] = b[k] - q[j] * b[j]
            for i in range(k-1,-1,-1):
                m[k][i] = np.inner(b[k], b_[i]) / np.inner(b_[i], b_[i])
        if B[k] >= (delta - np.power(m[k][k - 1], 2)) * B[k - 1]:
            k += 1
        else:
            b[k], b[k - 1] = b[k - 1].copy(), b[k].copy()
            tmp_b,tmp_m=gram_schmid(b)
            b_[k],b_[k - 1]=tmp_b[k],tmp_b[k-1]
            m[k],m[k - 1]=tmp_m[k],tmp_m[k-1]
            B = np.array([norma(i) for i in b_])
            k = max(1, k - 1)

        if (old_b == b).all() and (old_b_ == b_).all() and counter>2:
            return b
        else:
            old_b = b.copy()
            old_b_ = b_.copy()
            counter+=1
    return b

In [13]:
b_=LLL(V.copy())
b_

array([[ -6,   2,   0,   0,   0, -24],
       [-16,   6,   0,   0,   0,  17],
       [-32,  10,   2,   0,   0,   3],
       [ -1,   1,  -1,   1,  -1,   0],
       [  2,  -2,   0,   2,   0,  -4],
       [  0,  -2,   0,   0,   2,   2]])

In [14]:
smallest_v=sorted(list(b_),key=norma)[0] # наименьшая строчка по норме из полученного после LLL базиса
res=np.dot(smallest_v,np.linalg.inv(V)).astype("int")*-1
print("Результат взлома:",res[:-1])

Результат взлома: [1 0 1 0 1]


## Функциональная реализация

In [22]:
def encrypt(x,M):
    return(x * M).sum()
def decrypt(S,A,B,r):
    A_=invmod(A,B)
    S_=(A_*S)%B
    return find_x(r,S_)
def hak(M,S):
    V=np.array([[ 1 if i==len(M) else (2 if i==j else 0)  for j in range(len(M))] for i in range(len(M)+1)])
    last_col=np.array([S if i==len(M) else M[i] for i in range(len(M)+1)]).reshape(-1,1)
    V = np.hstack([V, last_col])
    b_=LLL(V.copy())
    smallest_v=sorted(list(b_),key=norma)[0]
    return (np.dot(smallest_v,np.linalg.inv(V)).astype("int")*-1)[:-1]

In [23]:
r_t = np.array((3, 11, 24, 50, 115)) # секретный ключ Алисы
A_t = 113 # секретный ключ Алисы
B_t = 250 # секретный ключ Алисы
M_t=np.array(list(map(lambda x: (x*A_t)%B_t,r_t))) # открытый ключ Алисы

x_t = np.array((1, 0, 1, 0, 1)) # сообщение

In [24]:
S_t=encrypt(x_t,M_t)
d_t=decrypt(S_t,A_t,B_t,r_t)
hak_t=hak(M_t,S_t)
print("Исходное:",x_t,"\nЗашифрованное:",S_t,"\nРасшифрованное:",d_t,"\nВзоманное:",hak_t)

Исходное: [1 0 1 0 1] 
Зашифрованное: 546 
Расшифрованное: [1 0 1 0 1] 
Взоманное: [1 0 1 0 1]
